### This code details the construction of the metabolic distance comparisons in Figure 2a-c and Extended Data Figure 5. 

The cells below create a dataframe that contains every pairwise distance for mega medium grown strains using the specified metabolomic distance matrices, the V4 region 16s, and the full length 16S. The also calculate common taxonomic ancestry and equivlance class.

In [ ]:
import pickle
# Create a tip renaming map so we can associate 16s features in the phylogenetic with their data
_fp = 'taxonomy_to_display_name.txt'
taxonomy_to_display_map = pd.read_csv(_fp, index_col=0, sep='\t')
tip_renaming_map = {t: d for t, d in zip(tmp.index, tmp['disp_name'])}

# Create tip data pickle
fp = '../supplemental_table_6.xlsx'
taxonomies = pd.read_excel(fp, index_col=0, sheet_name='full_taxonomy')

lvls = ['phylum', 'class', 'order', 'family', 'genus']

tmp = pd.read_csv('taxonomy_to_display_name.txt', index_col=0, sep='\t')
tip_data = {}
tip_translator = {}
for tip in tmp.index:
    tip_translator[tmp.loc[tip, 'disp_name']] = tip
    tip_data[tip] = {'disp_name': tmp.loc[tip, 'disp_name']}
    for lvl in lvls:
        tip_data[tip][lvl] = taxonomies.loc[tip, lvl]
#pickle.dump(open('./tipdata.p', 'wb'))

In [ ]:
# Normal utilities
import numpy as np
import pandas as pd

from scipy.cluster import hierarchy
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage

from skbio.tree import TreeNode

def _tlca(c1_levels, c2_levels):
    levels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus',
              'species', 'strain']
    for i in range(1, 8):
        if c1_levels[i] != c2_levels[i]:
            return i - 1
        else:
            pass
    return 7

# Output file path for everything.
BASE_OUT_FP = 'ward_metabolomics_linkage/'

###
# This part of the code actually generates the WL trees and colors them.
###

# Aggregated metadata
agg_md = pd.read_excel(io='../supplemental_table_7.xlsx', sheet_name='aggregated_md', index_col=0)

# Master sample database
md = pd.read_excel(io='../supplemental_table_5.xlsx', index_col=0, sheet_name='mf')

cpd_lib_fp = '../Supplementary_Table_1_mz-rt_library.xlsx'
ci = pd.read_excel(cpd_lib_fp, sheet_name='chemical_info', index_col=0)
chemical_info = pd.read_excel(cpd_lib_fp, sheet_name='chemical_info')

istds = ci.loc[['IS_' in i for i in ci['Compound']], :].index.values

# Taxonomic info
sac_fp = '../supplemental_table_6.xlsx'
taxonomies = pd.read_excel(sac_fp, index_col=0, sheet_name='full_taxonomy')

# Use only mega media samples.
mega_media_samples = ((agg_md['sample_type'] == 'supernatant') &
                      (agg_md['media'] == 'mm'))

ss_md = agg_md.loc[mega_media_samples, :]

datasets = [(1, 'count.ps'),
            (0, 'foldchange'),
            (0, 'foldchange.fa.ps'),
            (0, 'foldchange.dmrvf.fa.ps')]


tmp_results = {}

for _add_val, datafp in datasets:
    sup_data = pd.read_excel(io='../supplemental_table_7.xlsx', sheet_name=datafp, index_col=0) + _add_val
    
    data = sup_data.loc[mega_media_samples, :].copy()
    data = data.loc[:, ~np.in1d(data.columns, istds)]
    data['taxonomy'] = ss_md.loc[data.index, 'taxonomy']
    _cdata = data.groupby('taxonomy').agg('mean')

    #assert (_cdata.index == row_strains).all()

    for scale, cdata in zip(('linear', 'log'),
                            (_cdata, np.log2(_cdata))):

        nr, nc = cdata.shape
        results = np.zeros((nr, nr))
        #condensed_results = np.zeros(int(nr * (nr - 1) / 2))
        condensed_results = []

        for i in range(nr):
            v_i = cdata.iloc[i, :]
            v_i_mask = pd.notnull(cdata.iloc[i, :])
            for j in range(i+1, nr):
                v_j = cdata.iloc[j, :]
                v_j_mask = pd.notnull(cdata.iloc[j, :])

                shared_metabolites = v_i_mask & v_j_mask
                d = euclidean(v_i[shared_metabolites], v_j[shared_metabolites])
                results[i, j] = d

        tmp_results[(datafp, scale)] = results
        print(datafp, scale)

In [ ]:
row_strains = cdata.index.values.copy()

nr, nc = results.shape
rows = int(nr * (nr - 1) / 2)
cols = len(datasets) * 2
tmp_data = np.zeros((rows, cols))

strain_data = []

row = 0
for i in range(nr):
    strain_i = tip_renaming_map[row_strains[i]]
    for j in range(i+1, nr):
        strain_j = tip_renaming_map[row_strains[j]]
        col = 0
        for (_, datafp) in datasets:
            for scale in ('linear', 'log'):
                tmp_data[row, col] = tmp_results[(datafp, scale)][i, j]
                col += 1
        strain_data.append([row_strains[i], row_strains[j], strain_i, strain_j])
        row += 1

columns = ['c_lin', 'c_log', 'fc_lin', 'fc_log', 'fc_fa_ps_lin', 'fc_fa_ps_log',
           'dmrvf_lin', 'dmrvf_log']
r = pd.DataFrame(tmp_data, columns=columns)

columns = ['s1', 's2', 'tip_s1', 'tip_s2']
r2 = pd.DataFrame(strain_data, columns=columns)

final = pd.concat((r2, r), axis=1)

phylogenetic_tree = TreeNode.read('phylogenetic_tree.tr')
phylo_ttds = phylogenetic_tree.tip_tip_distances()

phylogenetic_tree_fl16s = TreeNode.read('mega_media_renamed_clustal_omega.tre')
phylo_ttds_fl16s = phylogenetic_tree_fl16s.tip_tip_distances()

tmp_v3v4 = np.zeros(final.shape[0])
tmp_fl16s = np.zeros(final.shape[0])
row = 0
for t1, t2 in zip(final['tip_s1'], final['tip_s2']):
    tmp_v3v4[row] = phylo_ttds[t1, t2]
    
    if (t1 in phylo_ttds_fl16s.ids) and (t2 in phylo_ttds_fl16s):
        tmp_fl16s[row] = phylo_ttds_fl16s[t1, t2]
    else:
        tmp_fl16s[row] = np.nan
    row += 1

final['v3v4_d'] = tmp_v3v4
final['fl16s_d'] = tmp_fl16s


same_exp = []
shares_exp = []
for idx, (s1, s2) in final.loc[:, ['s1', 's2']].iterrows():
    tmp1 = agg_md.index[agg_md['taxonomy'] == s1]
    tmp2 = agg_md.index[agg_md['taxonomy'] == s2]
    if (set(agg_md.loc[tmp1, 'experiment'].values) ==
        set(agg_md.loc[tmp2, 'experiment'].values)):
        same_exp.append(True)
    else:
        same_exp.append(False)

    if len((set(agg_md.loc[tmp1, 'experiment'].values).intersection(
            set(agg_md.loc[tmp2, 'experiment'].values)))) > 0:
        shares_exp.append(True)
    else:
        shares_exp.append(False)


final['same_exp'] = same_exp
final['share_exp'] = shares_exp
final['all_exps'] = True

levels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species',
          'strain']
_taxa_levels = {}
for taxon in tip_renaming_map.keys():
    _taxa_levels[taxon] = taxonomies.loc[taxon, levels]

tmp = []
for s1, s2 in zip(final['s1'], final['s2']):
    tmp.append(_tlca(_taxa_levels[s1], _taxa_levels[s2]))

final['tlca'] = tmp
df = final.copy()

In [ ]:
# `tlca` encodes the last common ancestor that taxa share in a given
# comparison. 0 indicates that are in different phyla, 6 indicates they are
# different only in their strain.
equivalence_classes = ['Bacteria', 'Phylum', 'Class', 'Order', 'Family',
                       'Genus', 'Species']
eqcs = []
subsets = []
for sid, row in df.iterrows():
    s1 = row['s1']
    s2 = row['s2']
    tlca = row['tlca']

    s1_phylum = taxonomy.loc[s1, 'phylum']
    s2_phylum = taxonomy.loc[s2, 'phylum']

    if tlca == 0:
        # These should be equivalent conditions
        assert s1_phylum != s2_phylum
        subset = 'All'
    else:
        assert s1_phylum == s2_phylum
        subset = s1_phylum

    eqc = equivalence_classes[tlca]

    eqcs.append(eqc)
    subsets.append(subset)

tmp = df.copy()
tmp['eqc'] = eqcs
tmp['subset'] = subsets
tmp.columns = ['bacteria1', 'bacteria2', 'tip_s1', 'tip_s2', 'c_lin', 'c_log',
                'fc_ling', 'fc_log', 'fc_fa_ps_lin', 'fc_fa_ps_log',
                'dmrvf_lin', 'metab_dist', 'short_16s_dist', 'long_16s_dist',
                'tlca', 'same_exp', 'share_exp', 'eqc', 'subset']

tmp.to_csv('distances.txt', sep='\t')

This cell creates Fig. 2b,c

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.nonparametric import smoothers_lowess as sl

# Make palette
colors = [plt.cm.plasma(i/7) for i in range(7)][::-1]
nested_taxonomy = ['Species', 'Genus', 'Family', 'Order', 'Class', 'Phylum',
                   'Bacteria']
palette = {i: j for i, j in zip(nested_taxonomy, colors)}


data_fp = 'distances.txt'
data = pd.read_csv(data_fp, sep='\t', index_col=0)
data.columns = ['bacteria1', 'bacteria2', 'tip_s1', 'tip_s2', 'c_lin', 'c_log',
                'fc_ling', 'fc_log', 'fc_fa_ps_lin', 'fc_fa_ps_log',
                'dmrvf_lin', 'metab_dist', 'short_16s_dist', 'long_16s_dist',
                'tlca', 'same_exp', 'share_exp', 'eqc', 'subset']

plotting_data = data[data['share_exp'] == True].copy()


plt.close('all')
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 10), sharey=True)

ax1.set_axisbelow(True)
ax1.grid(axis='y')
ax2.set_axisbelow(True)
ax2.grid(axis='y')

le = sl.lowess(plotting_data['metab_dist'].values,
               plotting_data['short_16s_dist'].values)
ax1.plot(le[:, 0], le[:, 1], alpha=1.0, color='k', linewidth=4,
         zorder=10)
_ = sns.scatterplot(data=plotting_data, x='short_16s_dist', y='metab_dist',
                    hue='eqc', palette=palette, linewidths=0, edgecolor='none',
                    ax=ax1)
ax1.legend().remove()

ax1.vlines(x=[0.11], ymin=[0], ymax=[70], lw=1, color='gray', linestyle='--')


ax1.set_yticks(np.arange(0, 70, 5))
ax1.set_yticklabels(np.arange(0, 70, 5), fontsize=20)
ax1.set_ylabel('Metabolomic distance', fontsize=24)

ax1.set_xlabel('Phylogenetic distance\n(V4 region 16s)', fontsize=24)
ax1.set_xticks(np.arange(0, .45, 0.075))
ax1.set_xticklabels(np.arange(0, .45, 0.075).round(3), fontsize=20)
ax1.set_xlim(0, 0.33)
ax1.set_ylim(0, 65)


_ = sns.boxplot(data=plotting_data, x='eqc', y='metab_dist', hue='eqc',
                 order=nested_taxonomy, hue_order=nested_taxonomy, ax=ax2,
                 boxprops={'fill':None, 'linewidth':2, 'color':'black'},
                 whiskerprops={'linewidth':2, 'color':'black', 'zorder':10},
                 medianprops={'linewidth':2, 'color':'black', 'zorder':10},
                 capprops={'linewidth':2, 'color':'black', 'zorder':10},
                 showfliers=False, dodge=False)


_ = sns.stripplot(data=plotting_data, x='eqc', y='metab_dist', hue='eqc',
                  ax=ax2, hue_order=nested_taxonomy, jitter=True,
                  dodge=False, zorder=0, palette=palette, alpha=1.0,
                  order=nested_taxonomy)
ax2.legend().remove()


ax2.tick_params(width=0, axis='y')

ax2.set_xticklabels(['Species', 'Genus', 'Family', 'Order', 'Class', 'Phylum',
                     'Kingdom'], rotation=90, fontsize=20)
ax2.set_xlabel('Last common rank', fontsize=24)
ax2.set_ylabel('')

plt.subplots_adjust(wspace=0.02, bottom=0.2)


fp = 'figure2bc.pdf'
fig.savefig(fp, transparent=True, dpi=300)
plt.close('all')

This cell does the permutation testing for Extended Data 5a.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

###
# Code for creating tip to tip distances
###
# _tree = TreeNode.read('phylogenetic_tree.tr')
# muscle_tree = _tree.root_at_midpoint()

# _tree = TreeNode.read('foldchange.dmrvf.fa.ps.txt.log.renamed_wl.tre')
# metab_tree = _tree.root_at_midpoint()

# tips1 = set([tip.name for tip in muscle_tree.tips()])
# tips2 = set([tip.name for tip in metab_tree.tips()])

# assert tips1 == tips2

# muscle_dm = muscle_tree.tip_tip_distances()
# metab_dm = metab_tree.tip_tip_distances()

muscle_dm = pd.read_csv('muscle_dm.txt', sep='\t', index_col=0)
metab_dm = pd.read_csv('metab_dm.txt', sep='\t', index_col=0)
tip_data = pickle.load(open('tipdata.p', 'rb'))
tip_translator = {v['disp_name']: k for k,v in tip_data.items()}

# 200 is an arbitrary constant here - just larger than the largest distance
# expected in either tree.
assert 200 > max(metab_dm.values.max(), muscle_dm.values.max())
diag = 200 * np.eye(muscle_dm.values.shape[0])

muscle_features = np.array(muscle_dm.index)
features = np.array(metab_dm.index)

assert (muscle_features == features).all()

tlevels = ['phylum', 'class', 'order', 'family', 'genus', 'species_', 'disp_name']

# same shape, r,c equal
r, c = np.triu_indices(metab_dm.shape[0], 1)
wl_dists = metab_dm.values[r, c]

r, c = np.triu_indices(muscle_dm.shape[0], 1)
phylo_dists = muscle_dm.values[r, c]

ntiles = [1, 2, 3, 4, 5, 10]
wl_ntiles = np.percentile(wl_dists, ntiles)
phylo_ntiles = np.percentile(phylo_dists, ntiles)

nrows, ncols = metab_dm.shape

tmp = []
results = []
for ntile_level, (wl_radius, phylo_radius) in enumerate(zip(wl_ntiles, phylo_ntiles)):
    metab_nk = (metab_dm.values + diag <= wl_radius)
    phylo_nk = (muscle_dm.values + diag <= phylo_radius)

    for row in range(nrows):
        metab_features = features[metab_nk[row]]
        phylo_features = features[phylo_nk[row]]

        if (metab_features.sum() == 0) or (phylo_features.sum() == 0):
            pass
        else:
            for lvl in tlevels:
                # Features are in the same order so we only have one set of labels.
                mfeatures_at_lvl = np.array([tip_data[tip_translator[i]][lvl] for i in metab_features])
                pfeatures_at_lvl = np.array([tip_data[tip_translator[i]][lvl] for
                                             i in phylo_features])

                feature_union = np.array(sorted(set(mfeatures_at_lvl).union(set(pfeatures_at_lvl))))

                m_count = np.zeros(len(feature_union))
                p_count = np.zeros(len(feature_union))
                for idx, f in enumerate(feature_union):
                    m_count[idx] = (mfeatures_at_lvl == f).sum()
                    p_count[idx] = (pfeatures_at_lvl == f).sum()


                c = np.vstack((m_count, p_count)).min(0).sum()
                max_possible = min(m_count.sum(), p_count.sum())
                fractional_c = c / max_possible

                perm_count = 0
                for iter in range(1000):
                    n_perm_metab_features = len(metab_features)
                    n_perm_phylo_features = len(phylo_features)
                    _fm = np.arange(ncols)
                    np.random.shuffle(_fm)
                    fm = features[_fm[:n_perm_metab_features]]

                    _fp = np.arange(ncols)
                    np.random.shuffle(_fp)
                    fp = features[_fp[:n_perm_phylo_features]]

                    # Features are in the same order so we only have one set of labels.
                    mfeatures_at_lvl = np.array([tip_data[tip_translator[i]][lvl] for
                                                 i in fm])
                    pfeatures_at_lvl = np.array([tip_data[tip_translator[i]][lvl] for
                                                 i in fp])

                    feature_union = np.array(sorted(set(mfeatures_at_lvl).union(set(pfeatures_at_lvl))))

                    m_count = np.zeros(len(feature_union))
                    p_count = np.zeros(len(feature_union))
                    for idx, f in enumerate(feature_union):
                        m_count[idx] = (mfeatures_at_lvl == f).sum()
                        p_count[idx] = (pfeatures_at_lvl == f).sum()

                    perm_c = np.vstack((m_count, p_count)).min(0).sum()
                    max_possible_perm_c = min(m_count.sum(), p_count.sum())
                    perm_fractional_c = perm_c / max_possible_perm_c

                    if fractional_c < perm_fractional_c:
                        perm_count += 1

                results.append([features[row], ntile_level, wl_radius, phylo_radius,
                                len(metab_features), len(phylo_features),
                                len(feature_union), lvl, c, max_possible, fractional_c, perm_count])

    print(row)

data = pd.DataFrame(results, columns=['feature', 'ntile_level', 'wl_radius', 'phylo_radius',
                                      'n_metab', 'n_phylo', 'n_union', 'level',
                                      'c', 'max_possible', 'fractional_c', 'perm_c'])
#data.to_csv('./extended_data_5a/extended_data_5a.txt', sep='\t')

This cell creates the plot for Extended Data 5a.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

out_fp = './extended_data_5a/'
xlabels = ['Strain', 'Species', 'Genus', 'Family', 'Order', 'Class', 'Phylum']
levels = ['disp_name', 'species_', 'genus', 'family', 'order', 'class', 'phylum']
for xlabel, level in zip(xlabels, levels):
    plotting_data = data[(data['ntile_level'] == 3) &
                         (data['level'] == level)]

    xbins = [0, 50, 1000]
    ybins = np.linspace(0, 1, 6)

    xbinticklabels = ['p<=0.05', 'p>0.05']
    ybinticklabels = ['%s-%s' % (ybins[i].round(2), (ybins[i] + 0.2).round(2))
                      for i in range(5)]

    c, x, y = np.histogram2d(plotting_data['perm_c'],\
                             plotting_data['fractional_c'], 
                             bins=(xbins, ybins))

    fig = plt.figure(figsize=(2, 10))
    ax = fig.add_subplot(111)
    sns.heatmap(c.T, square=True, xticklabels=xbinticklabels,
                yticklabels=ybinticklabels,
                annot=True, ax=ax, cbar=None, cmap=plt.cm.Reds,
                annot_kws={'fontsize':20})
    ax.set_title('%s' % xlabel, fontsize=20)
    plt.subplots_adjust(left=0.55, bottom=0.25)
    plt.setp(ax.get_xticklabels(), fontsize=20, rotation=90)
    plt.setp(ax.get_yticklabels(), fontsize=20, rotation=0)
    plt.savefig(out_fp + xlabel + '.pdf', dpi=300, transparent=True)
    plt.close('all')

sns.heatmap(c, annot=True)

This cell demonstrates the linear models used in Supplementary Table 7 tab "regression_results".

In [ ]:
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd

data = pd.read_excel(io='../supplemental_table_7.xlsx', sheet_name='distances', index_col=0)
data.columns = ['bacteria1', 'bacteria2', 'tip_s1', 'tip_s2', 'c_lin', 'c_log',
                'fc_ling', 'fc_log', 'fc_fa_ps_lin', 'fc_fa_ps_log',
                'dmrvf_lin', 'metab_dist', 'short_16s_dist', 'long_16s_dist',
                'tlca', 'same_exp', 'share_exp', 'eqc', 'subset']
# Stats models sorts all the levels of a categorical variable and uses the
# first entry (lexographically sorted) as the base case.
data.loc[(data['eqc'] == 'Species').values, 'eqc'] = 'AAA_Species'
data_1 = data[data['share_exp'] == True]

# Example of distance constraint
# data_1 = data[(data['share_exp'] == True) &
#               (data['short_16s_dist'] >= 0.11)]

f = 'metab_dist ~ eqc'

q = smf.ols(f, data=data_2).fit()
print(q.summary())